# Tranformers

이전 장에서는 Transformer 아키텍처가 두각을 나타낼 때까지 NLP에서 유행하는 모델링 아키텍처인 RNN을 다루었습니다.

트랜스포머는 최신 NLP의 핵심입니다. 2017년에 처음 제안된 원래 아키텍처는 (딥 러닝) 세계를 폭풍으로 몰아넣었습니다. 그 이후로 NLP 문학은 세서미 스트리트 문자(Sesame street characters) 또는 "-former"로 끝나는 단어로 광범위하게 분류되는 모든 종류의 새로운 아키텍처로 범람했습니다.

이 장에서 우리는 바로 그 아키텍처, 즉 트랜스포머를 자세히 살펴볼 것입니다. 우리는 핵심 혁신(the core innovations)을 분석하고 신경망 계층의 새로운 범주인 주의 집중 메커니즘(the attention mechanism)을 탐구할 것입니다.

## Building a Transformer from Scratch

2장과 3장에서는 변환기를 실제로 사용하는 방법과 사전 훈련된 변환기를 활용하여 복잡한 NLP 문제를 해결하는 방법을 살펴보았습니다. 이제 아키텍처 자체에 대해 자세히 알아보고 변환기가 첫 번째 원칙에서 작동하는 방식을 알아봅니다.

"제1원칙"은(는) 무슨 뜻인가요? 음, 우선 Hugging Face Transformers 라이브러리를 사용할 수 없음을 의미합니다. 우리는 이미 이 책에서 그것에 대해 많이 열광했습니다. 그래서 우리는 그것에서 잠시 휴식을 취하고 내부에서 실제로 어떻게 작동하는지 볼 때입니다. 이 장에서는 원시 PyTorch를 대신 사용할 것입니다.

대부분의 연구자가 사용하는 본격적인 딥 러닝 라이브러리인 PyTorch는 자연스럽게 Hugging Face 라이브러리와 마찬가지로 매우 인기 있는 변환기 아키텍처를 구현합니다. 그러나 nn.Module로 노출되는 이 버전은 DIY에 훨씬 더 가깝고 데이터 로더, 옵티마이저 등과 같은 다른 친숙한 PyTorch 도구와 함께 사용하기 위한 것입니다.

앞에서 언급했듯이 딥 러닝 관련 클래스/함수가 수행하는 작업을 확인하는 가장 좋은 방법 중 하나는 유형 서명(type signature)과 입력 및 출력의 차원을 살펴보는 것입니다. 그렇게 해봅시다:

In [ ]:
import torch

In [ ]:
model = torch.nn.Transformer()
model.encoder.layers[0]

In [ ]:
TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
        (out_proj): Linear(in_features=512, out_features=512, bias=True)
    )
    (linear1): Linear(in_features=512, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=512, bias=True)
    (norm1): LayerNorm(512,), eps=1e-0.5, elementwise_affine=True)
    (norm2): LayerNorm(512,), eps=1e-0.5, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
)    

처음에는 이것에서 빼낼 것이 너무 많지 않은 것 같습니다. 우리를 위해 정의된 표준 forward() 함수가 있는 상당히 표준적인 PyTorch nn.Module입니다. 원칙적으로 우리는 그것을 훈련 파이프라인에 연결하고 계속할 수 있습니다. 사실 이것은 본질적으로 우리가 2장에서 했던 것입니다. 하지만 이 모듈의 정확한 구성 요소를 이해하려고 노력합시다.

특히 흥미로운 것은 MultiHeadAttention 계층입니다. Dropout, Linear 및 LayerNorm과 같은 대부분의 다른 레이어는 비변환기 모델에서도 볼 수 있을 것으로 예상할 수 있습니다. PyTorch에 의한 변환기의 이 특정 구현(추가 구성 매개변수 없음)은 공교롭게도 제목이 "Attention Is All You Need"인 원본 논문(그림 7-1 참조)의 아키텍처입니다.

요컨대, 이 Transformer 클래스의 가장 중요한 구성 요소는 MultiHeadAttention 계층이라고 해도 무방합니다. 따라서 그것이 무엇이며 어떻게 작동하는지 이해하는 데 시간이 걸리는 것이 합리적입니다.

## Attention Mechanisms

주의 집중(attetion) 메커니즘은 심층 신경망의 계층입니다. 여전히 해석의 여지가 있는 작업이지만 장거리 "전역" 기능을 학습하는 것입니다. 어텐션 메커니즘은 임베딩 벡터의 입력 시퀀스에서 단일 출력 벡터에 기여하는 구성 요소를 결정하는 "정보 라우터"라고 부르는 역할을 합니다. 이 아이디어는 우리가 실제로 세부 사항을 작업하면서 더 명확해질 것입니다.

우리는 작년에 NeurIPS에 참석한 다른 수천 명의 사람들과 마찬가지로 관심(attention)에 대해 이야기하게 되어 기쁩니다. 하지만 그 전에 주목해야 할 중요한 주제는 관련된 작업의 계산적 복잡성이라는 점을 언급해야 합니다. 표시되고 관련된 텐서의 크기에 대해 내적/행렬 곱셈의 수를 생각해 보세요.

### Dot Product Attention

좋아요, 엄밀히 말해서 우리는 이러한 유형의 주의 집중(attention)이 실제 네트워크에 실제로 적용되는 것을 본 적이 없다고 생각합니다. Scaled dot product Attention은 일반적으로 다음에 논의할 Multi-Head Self-Attention의 구성 요소로 언급됩니다.

이국적인 관심(attention) 메커니즘의 세계에서 당신이 해야 할 가장 중요한 질문은 이것입니다: 당신은 사물들 사이의 유사성을 정확히 어떻게 측정합니까? 선형 대수와 GPU의 버킷 로드에 가려진 이 핵심 아이디어는 오늘날 NLP에서 신경망의 기본 동작을 주도하는 것입니다.

그리고 스케일링된 내적은 유사성을 측정하는 가장 단순하고 직관적인 방법 중 하나인 내적을 사용합니다.

당신은 이것에 익숙해야 하지만 간단히 요약해 봅시다. 내적은 두 벡터를 취해 요소별로 곱한 다음 결과를 더하는 연산입니다. 이것은 우리가 "내적"하는 두 벡터가 유사한 구성 요소를 가지고 있으면 해당 요소의 곱이 크고 그 반대의 경우도 마찬가지이기 때문에 유사성을 측정합니다(유사하지 않은 구성 요소가 있는 벡터는 내적이 작다는 점에서).

하지만 진짜 질문은 우리가 내적을 취하는 것이 정확히 무엇입니까? 이 질문에 답하기 위해 이러한 어텐션 메커니즘이 변환기에서 구현되는 방식에 초점을 맞추겠습니다(그림 7-1 참조).

![image](images/transformer_layer.png)

일반적인 변환기는 일련의 단어 벡터를 입력으로 사용하고 각 레이어에서 이를 숨겨진 표현/상태라고 하는 벡터의 다른 시퀀스로 변환합니다(아니요, 그렇게 이름이 붙은 것 같지는 않습니다).

따라서 네트워크의 각 숨겨진 레이어에는 "참석(attent)"하려는 벡터 시퀀스가 있습니다. 그림 7-2를 참조하십시오.

![image](images/dotproduct_1-768x412.png)

자, 여기서 중요한 부분이 있으니 집중하세요(매우 의도된 말장난).

우리가 할 일은 숨겨진 상태 벡터 각각을 3개의 완전히 독립된 벡터(쿼리, 키 및 값)로 변환하는 것입니다.

간단한 행렬 곱셈을 통해 이 변환을 수행하고 이러한 벡터의 차원은 우리에게 달려 있습니다. 유일한 제한은 쿼리 벡터와 키 벡터가 동일한 차원을 가져야 한다는 것입니다(둘 사이에 내적을 수행할 것이므로).:

그런 다음 모든 키 벡터와 마찬가지로 각 시간 단계에서 쿼리 벡터의 내적을 취하여 주의 가중치를 계산하고 그 결과를 softmax합니다. 모든 시간 단계에서 동시에 이 작업을 수행하려면 곱셈을 병렬로 수행하는 행렬에 이러한 벡터를 압축하는 것이 더 효율적입니다. 최종 계산은 다음과 같습니다.:

하지만 그게 우리가 할 수 있는 전부는 아닙니다. 각 쿼리 벡터는 독립적이므로 시간에 따라 병렬화할 수 있습니다.:

하지만 왜? 그것이 트랜스포머 페이퍼를 읽을 때 우리가 스스로에게 물었던 질문입니다. 세 개의 벡터로 나누는 것은 약간 임의적이고 복잡해 보입니다. 예를 들어, 왜 두 개나 네 개가 아닌가?

이름만 보고 직감이 데이터베이스에 뿌리를 두고 있는 것 같습니다. 이것이 신경망 없이 일반적인 오래된 Python 사전(dictionaries)에서 어떻게 작동하는지 생각해 보세요.

대규모 키-값 쌍 시퀀스가 있습니다. 그것이 사전 구조입니다. 다음과 같이 보일 수 있습니다.:

```
sentence = {
    "word_1": "Squirtle"
    "word_2": "is"
    "word_3": "the"
    "word_4": "greatest"
    "word_5": "Pokemon"
    "word_6": "ever"
}
```

이제 나는 당신이 무슨 생각을 하고 있는지 압니다. "푸-리스! 우리 모두는 Squirtle이 Charizard에 대항할 기회가 조금도 없다는 것을 알고 있습니다."

글쎄요, 우리는 다를 것을 간청합니다. 받아 들여.

사전에서 값을 얻으려면 다음과 같은 쿼리를 사용합니다.:

```
a = sentence['word_3']
```

그리고 뒤에서 파이썬이 하는 일은 word_3 쿼리를 문장 사전에 있는 모든 가능한 키와 비교하는 것입니다. 그런 다음 값을 반환하고 변수에 저장합니다.

우리가 dot product Attention으로 하고 있는 것은 비슷합니다. 쿼리 벡터는 추상적인 의미에서 현재 단어가 찾고 있는 것을 나타냅니다. 각 단어 종류와 관련된 키는 각 단어가 제공해야 하는 것을 나타냅니다. 값 벡터에는 쿼리 벡터가 찾고 있던 정보가 포함됩니다. 그러나 우리는 그것이 매우 추상적으로 들린다는 것을 알고 있으므로 예를 보여 드리겠습니다.

다음 문장을 고려하십시오.:

>Mario is short, but the he can jump super high.

이제 변환기가 현재 "he"라는 단어에 대해 작업 중이고 이를 네트워크의 다음 계층으로 전파하려고 한다고 가정해 보겠습니다. 여기서 쿼리 벡터는 대명사 "he"가 정확히 무엇을 가리키는지 명확히 하기 위해 이름이나 사람을 찾는 것일 수 있습니다. 따라서 변환기는 "he"에 대한 쿼리 벡터를 사용하고 문장에서 다른 모든 단어의 키 벡터를 사용하여 이 쿼리 벡터의 내적을 계산합니다. 이러한 각 내적은 쿼리와 키가 일치하는 정도를 측정하는 일종의 정렬 소스를 생성합니다.

이렇게 하면 "Mario"에 해당하는 키 벡터가 켜지고 가장 큰 정렬 점수가 생성됩니다. 이것은 네트워크에 흥미로운 일이 진행되고 있음을 나타내며 네트워크는 주의를 기울여야 합니다(우리가 거기서 무엇을 했는지 보십시오).

그러나 작업은 아직 완료되지 않았습니다. 변환기가 "he"와 문장의 다른 단어 사이의 모든 정렬 점수를 계산하면 이 점수를 softmax를 통해 전달하여 멋진 분포를 생성합니다. 점수를 더 자연스럽게 해석할 수 있습니다. 0은 단어 사이에 연관성이 거의 없다는 것을 의미하고 1은 거의 완벽하게 일치한다는 의미입니다.

각 단어에는 관련 값 벡터가 있으며, 이 벡터는 Python 사전의 값과 마찬가지로 세상의 실제로 의미 있는 콘텐츠를 나타내는 것으로 간주됩니다. 그러나 Python 사전과 달리 각 쿼리는 단일 결과를 반환하지 않습니다. 대신 변환기는 각 단어에 대해 계산한 정규화된 정렬 점수를 가져와 모든 값 벡터에 대해 가중 합계를 수행하는 데 사용합니다. 그 이유는 다소 간단합니다. 예를 들어 우리가 작업하고 있던 문장은 "Mario와 Luigi는 키가 작지만 매우 높이 뛸 수 있습니다."였습니다. 여기서 "그들"에 대한 쿼리는 단일 단어를 찾는 것이 아니라 이 그룹에 맞는 가능한 모든 사람을 찾는 것입니다.

이제 정렬 점수 분포를 생성하면 문장의 다른 부분을 다른 양으로 선택할 수 있습니다! 약간 단순화하면 "Mario" 및 "Luigi"라는 단어의 정규화된 정렬 점수가 0.5이고 다른 모든 단어의 경우 0이라고 상상할 수 있습니다.

변환기는 이제 문장에 참석하고(진지하게도 단어인가요?) 특정 단어(이 예에서는 "he" 및 "they")에 대한 벡터를 생성했습니다. 큰 틀에서 이 단어와 관련이 있습니다.

이제 문장의 모든 단어에 대해 이 프로세스를 반복하여 네트워크의 더 깊은 레이어로 전달할 또 다른 벡터 시퀀스를 얻습니다.

트랜스포머의 인코더 부분에서 이른바 "셀프 어텐션"을 계산할 때 다음과 같은 숨겨진 상태가 계산에 사용됩니다.:

이들은 모두 인코더의 해당 계층에 있는 시퀀스에서 가져옵니다. 디코더의 self-attention도 마찬가지입니다.

쿼리에 대한 디코더 숨겨진 표현과 키 및 값에 대한 인코더 숨겨진 표현을 사용하는 디코더에 사용되는 주의 집중(attention) 계층이 있습니다. 이를 통해 디코더는 이전의 모든 인코더 숨겨진 표현을 처리할 수 있으며 이는 기계 번역과 같은 작업에 유용합니다. 프랑스어 번역가가 실제로 전체 문장을 먼저 영어로 읽지 않고 횡설수설을 토해내는 것을 원하지 않을 것입니다.

그림 7-2에서 self-attention 계층을 시각화하고 그림 7-1에서 전체 계층을 함께 시각화할 수 있습니다. 이 레이어 몇 개를 다른 레이어 위에 쌓으면 붐! 당신은 (거의) 변압기(transformer)를 얻었습니다.

### Scaled Dot Product Attention

There's a minor problem with this, though. Although dot products are really fast, cool, and all that, when the size of the vectors are large, the dot product can get pretty big.

To see what we mean, consider two random vectors. Instead of just talking about it, though, let us show you some actual computations from NumPy:

In [ ]:
import numpy as np

In [ ]:
small_dots = [
    np.dot(np.random.randn(10),
           np.random.randn(10))
    for i in range(100)]
np.mean(np.absolute(small_dots))

What we just did there was generate two random arrays of size 10 and take the dot product of them. Just to be sure, we repeated this 100 times and calculated the average magnitude of the dot product to make sure that we're not getting a random outlier.

And so what the value was around 2.74. How's that usefull? Well, let's try the same thing with arrays of size 10,000:

In [ ]:
large_dots = [np.dot(np.random.randn(10000),
                     np.random.randn(10000))
              for i in range(100)]
np.mean(np.absolute(large_dots))

Ok. That's a lot bigger. But think about it-since we're using dot products to measure alignment, something is clearly wrong here. In both cases, we generated purely random vectors, so ideally, their alignment scores should be similar.

But since the components are chosen from a standard normal distribution with mean 0 and variance 1, an n-dimensional vector will have a variance of n (you get his by adding up the variances of the components, and if you're going to be pedantic, it's the trace of the covariance matrix of the vector, but that's way too long a name).

To correct for this and ensure that vectors of any dimensionality have roughly the same alignment scores, we'll scale our previous attention mechanism similar to how you'd normalize to unit variance in statistics.

The new, corrected attention mechanism would be:

### Multi Head Self Attention

Here's something you might find interesting: the two attention mechanisms that we just discussed, and the one we're about to show you now, all came from the same paper-"Attention Is All You Need" (aka the transformer paper). Pretty cool, huh?

Anyway, the next thing we can do is try to split up our attention mechanism into many smaller attention mechanisms (with an "s"). Why would wnat to do this? A good way to illustrate the rationale is through a popular attention test video (https://youtu.be/vJG698U2Mvo)

Now you've probably seen that video before (if you haven't, surprise!), and you know why it's so hard to spot the gorilla on your first pass-it's easier and more national to pay attention to one thing at a time. In this case, that's baskeball passes, since that's what the video asks you to look for. If instead, you were asked to look for a gorilla, it probably would have been easier to find the gorilla.

Attention mechnisms kind of work in the same way. There's a lot of stuff to pay attention to and keep track of in language, like pronouns, as we discussed earlier ("Mario is short, but he can jump super high"), but also other things, like where the main characters are going in physical space ("Mario went to the flower store and then to the gym, where he did 50 squats")

Having one set of queries, keys, and values do all that work might be a bit too much, and they might miss out on the occational gorilla, just like you probably did.

Multi-head attention mechanisms try to fix this issue by independently applying the attention mechanism multiple times on the same sequence in a single pass. In terms of the gorilla video, this would be like having your buddy watch the video with you. One of you could pay attention to the passes, while the other could look for gorillas, thereby increasing the overall attention capablilities.

Crucially, the query key and value matrices need to be different, otherwise redoing the whole attention thing multiple times would just be a waste of computation (asking your friend to look for passes while you also look for passes).

To create variety in the queries, keys, and values, the transformer network simply uses multiple sperate weight matrices to transform the input into multiple queiries, keys, and values:

Here, n is the parameter you set, and it's called the number of heads. It represents how many different attention computations are being done on the same sequence. You can think of it as the number of people you invite over to watch that gorilla video with you.

Are you tired of that analogy yet? Don't worry, we're almost done with it.

Each one of these "heads" performs the scaled dot product attention calculation independently (and, crucially, in parallel):

At the end of all this number-crunching, we're going to be left n different output vectors per spot in the sequence, corresponding to the outputs from each of the attention heads. But since the next layer needs a sequence of vectors (and a sequence of n vectors), the transformer concatenates the output from the multiple attention heads and passes it through another learned linear transform to make the dimenstions work right:

A sequence of these new concatenated and transformed z vectors is what gets passed on to the next layer of the transformer.

That's difinitely a lot of linear algebra to take in at once, so go through it slowly again to make sure you actually get it. In particular, visualizing Multi-Head Self-Attention is probably the best way to understand how it works. Jay Alammar has an exellent set of article on this (https://jalammar.github.io/illustrated-gpt2/), and we hightly encourage you to take a look at the viusalizations presented there.

### Adaptive Attention Span

OK, we're finally moving on to some (relatively) newer and cooler stuff. In 2019, some cool people at Facebook AI Research asked a really cool question-what if we could get transformer to learn what to pay attention to?

But isn't that what transformers aleardy do? Isn't this the entire point of the attention mechanism?

Well, yes. But there's also another very important thing we haven't talked about-computaional cost. You see, adding an attention mechanism isn't cheap. If you have n words in a batch/sentence, it would taken $n^2$ dot products (per layer) to compute each of the attention weights across all the tokens in the sequence. This is because you have to take the dot product of each of the $n$ query vectors with each of the n keys.

As you can tell, this can blow up pretty fast. If you had, say, 50 takens/words in your batch/sentence, then there are at least $50^2 = 2,500$ dot products to compute. But simply by increasing the number of tokens by two, to 52, you'd now have more than $52^2 = 2704$ dot products to compute. That's about 200 more dot products justfor adding two extra tokens per batch, and that's not even factoring in multi-headed attention!

Of course, one could question if we really need to compute attention over every single token every single time. It sees a little excessive. Especially in character-level or subword-level models, where some of the attention heads might simply be looking at the last few tokens to try and fit characters or subwords together into words. But then make every head look at only the last few tokens.

The way we (or in this case, the Facebook team; we are just reaping the benefits of their work) strike a balance is by having some heads attend over a larger set of tokens, and have some heads attend over only the last few tokens.

There's one term we'll introduce here: attend span. This simply refers to how many previous tokens the model is attending to. So if a head has an attention span of 5, this means that head runs an attention mechaniszm over the last 5 tokens from the current position in the sequence.

So how do we decide the attention span for each of the heads? Typically, this would involve experiments, plots, some hand-waving, and a fair bit of guesswork. But what makes adaptive attention span so cool is that each head can learn its own attention span though the training process!

This idea is really cool because it takes something that would have been a hyperparameter, the number of tokens to attend over, and makes it a simple parameter that can be automatically tuned through backprop.

Here's the main issue at hand: the number of tokens that each head looks at, also called the attention space, is an interger, and therefore can't be differentiated. Being nodifferentiable means that you can't really learn that parameter though training. So instead, the research team had to come up with a clever way to get a differentiable version of the attention span.

They did this by creating something called a masking function, which takes in th distance between tokens and outputs a value between 0 and 1. In the paper, they define the masking function like this:

Which we guess looks a little weird. But the plot is actually pretty clear and simple, as shown in Figure 7-3.

So the intuition here is that if the distance x between two tokens is large enough, the value of $m_z(x)$ will be zero, which means we don't do the attention computation between those two tokens.

Since this $m_z(x)$ fuction is smooth, we can get its gradient and tune the value of $z$ for each attention head. With a larger $z$, the attention head would look across more tokens, and vice versa. $R$ is a hyperparamter that controls the smoothness of that ramp section you see on the plot.

But most importantly, the adaptive attention span transformer has some pretty cool results. It achives state-of-the-art performance on the enwik8 dataset using considerably less memory and FLOPs than other transformers.

### Persistent Memory/All-Attention

This modification to the self-attention mechanism is a little interesting, because it focuses on something that deep learning research rarely does-simplicity.

The all-attention layer, introduced in a paper by FAIR (yes, those same people again) doesn't significantly improve performance or decrease computational cost. Instead, it takes a multistep process in the original Transformer architecture and reformulateds it into a single step that involves just the attention mechanism, nothing else.

In the original implementation, the transformer uses a postion-wise feed-foward network in each layer. What this means is that after running the attention mechanism, the transformer passes each of the vectors in the sequence through a tiny vanilla neural net before passing it on to the attention mechanism in the next layer.

And here's the juicy bit-the persistent memory paper says that most of the parameters from the transformer are used in these feed-forward networks, not the attention and self-attention mechanisms.

So their idea was to get rid of the postion-wise feed-forward network entirely. Not necessarily to reduce the number of parameters (because they endup adding back in a lot more parameters eventually), but just because.

They showed that if you stare at the computation of the postion-wise feed-forward networks, it actually looks similar to the computation that an attention mechanism is doing. Let's take a look and see what the authors mean:

where $U, V$ in the feed-forward network are weight matrics.

Don't really see the connection between the two? Yeah, neither do we. But take a look at what happens when we remove the bias terms and swap out the ReLU for a softmax:

Now if you look carefully at the last step, you'll notice that what we're doing is a matrix-vector product between $U$ (the matrix) and $Vx_t$ (the vector). And you remember the details of how tht works-this is basically taking a weighted sum of the columns of the $U$ matrix:

where the attention weights $a_ti$ are computed from the $Vx_t$ product and $u_i$ is the ith column of $U$.

Looking at the computations in this way, $x_t, V, U$ are analogous to the queries, keys, and values in scaled dot product attention.

So what's the point of all this math, you ask? Well, actually not much. Sorry.

The main conclusion of the paper is that since the computation that the feed-forward networks are doing is very similar (in fact, almost equivalent if you ignore the bais terms and activation function), we can probably swap them out and make the transformer architecture simpler.

In our opinion, it would have been equally valid to ust say, "Hey look, so we got rid of those feed-forward network things after the attention mechanism and just used a bunch of attention instead. It worked pretty well." But hey, it what it is.

If you're beginning to question the meaning of life ater spending the last five minutes of your precious free time breaking your head over a bunch of equations that we just told you do pretty much nothing new, fear not. Because this paper did have another really cool idea: persistent memory.

Considering that replaceing the feed-forward networks with attention would reduce the number of parameters in the model, the authors benevolently decided to not let their GPU memory get too bored, so they found some new ways to crank up the temperature on their Nvidia home heaters.

Now of course, if you wanted to add more parameters to your model, you could always do something simple, like increasing the number of layers, increasing the context size, etc. But instead, this FAIR team dicided to do something very clever. They decided to give the model an independent memory bank.

We'll be specific. When we say "memory bank", we mean a large collection of key-value vector pairs. You can have as many of these key-value vector pairs as you want, and they are completely independent from the actual training data.

Once you have this large bank of vectos, you can choose to run the attention mechanism over these vectos as well, not just the sequence from the text data. These vectors are then updated over the course of training, and used in the attention mechanism at inference time as well.

The key-value vectors then act as a sort of indexed knowledge base. If a transformer language model is trying to predict the next word of the sentence "World War II ended in," it would have a query for the next postion that corresponds to asking for the year that the Second World War ended. However, this information is nowwhere to be found anywhere else in the sentence, so the model just kind of has to guess.

But with a dedicated memory bank, the transformer can store all sorts of little tidbits like that, and when the query vector hits the right key in the memory bank, it can access the right information in it.

A few technical details for those of you who care: the positional embedding for the memory vectors are zero, and the keys and values are stacked into a matrix and concatenated onto the sequence for running the attention mechanism.

The idea of a dedicated memory unit in a neural ent isn't exactly new. But it's the idea of using a persistent memory bank as a way to inject more parameters into a relatively simple neural net architecture that makes this attention mechanism interesting.

### Product-Key Memory

Let's dive down the memory-augmented attention rabbit hole a bit further, since it seems to be a thing that's getting more popular, at least in the deep learning literature.

This next attention mechanism + memory unit that we're going to look at doesn't seem that cool if you look at it on its own. But it's actually used in XLM and CTRL, two state-of-the-art transformers that came out after this layer was introduced.

By the time this paper was published, memory in transformers was already a thing. So the goal of this project was to make memory more efficient.

It starts off with a very similar premise to the previous memory mechanism we talked about. We have a memory bank that consists of a large collection of key-value pairs, where the keys and values are both vectors.

In persistent memory, we attend over the entire memory bank, which can quickly blow up if the memory bank gets too big (which wasn't a super-big deal last time since that ahtuers were mostly trying to use memory to substitute for parameters lost in the feed-forward networks). Here, the anthors propose a different solution.

Instead of attending over a huge memory bank, most which will be pretty useless for each query, why not just pick a few keys and use the corresponding value? Specifically, they suggest finding the top k keys that maximize the dot product with the query, and using a weighted sum of the $k$ corresponding values to get a result from the memory bank, as shown in Figure 7-4. 

This seems neat, but it gets even cooler. Consider the case when you have a large number of keys; you then have to compute a lot of dot products, because you need to dot-product the query with each key to get the similarity scores before picking the top keys.

To make the top-$k$ key search more efficient, we can split each of the keys into half, so that instead of having, say, a key as a vector/arrayu with 10 elements, you can have 2 keys with 5 elements each.

Now if you pull out your old undergrad combinatorics textbook, you'll see that if you have n half-keys, and consider a full key to be the concatenation of two half-keys, then in total, you can make up to $n^2$ keys. An example with 3 subkeys is shown in Figure 7-5. What this means is that for a memory bank of $n^2$ value vectors, all we need is $n$ half-keys!

Using the power of half-keys, figure 7-6 shows how we'd now access values in the memory bank.

Now, let's break down what that diagram is saying. first, you split the query vector into two parts. With the first half of the query vector, you dot product it with al the first-half parts of the half-keys, and pick the top $k$ subkeys. Do the same for the bottom-half subkeys.

Now, since you picked $k$ subkeys for the first half, and $k$ subkeys for the bottom half, you'll end up with $k^2$ full-keys to pick from.

Now, instead of having a huge number of keys to search through, we just have $k^2$. So we compute the dot product between the query and these $k^2$ keys, but this time we use the full query and keys. Here, we're assumming that $k^2$ is much smaller than the full memory bank size, so this is actually still much more efficient than searching through every single one of the full-keys.

From there on out, it's just the standard scaled dot-product attention computation. Attend over those $k$ memory units that you just selected, and you've got yourself a super-efficient memory module to plug into your transformer.

But is all this effort worth it? How well does this half-key memory method work in practice? Well, according to the paper, they were able to beat a 24-layer BERT using just 12 layers + memory. So we'll leave it up to you to decide.

Here, we provided you a samll set of variants on the traditional attention mechanism that we found interesting. but this list is by no means complete, and the interest around attention mechanisms is at an all-time high. the Google Trends results for terms like "attention mechanism" and "transformers" look much more like stock prices during a bull run than the search frequency of scientific literature.

As the transformer architecture exploded in popularity, the entirety of the deep learning research community decided to go bullish on it, and since the release of the original transformer in 2017, the field has seen a influx of new variants on the Transformer architecture that promise to be more efficient, scale better, have a lower memory cost, etc. Today, there are more transformer-like architectures than we can possibly hope to include in one book, and new ideas keep pouring in on a weekly basis.

Hopefully this gives you the impression that transformers are not one single monolithic model that will be etched into stone walls like the fundamental equations of physics. Today, there are more variants than we can count, and it seems like there will soon be more variants than we can count, and it seems like there will soon be more variants than we can possible hope to name. Linformer, Longformer, Reformer, Performser, and Perceiver are just a few of the many new variants of the original Transformer architecture that are rapidly eating up the English language vocabulary.

Navigating this architectureal landscape is hard. Many times, research papers ptich their idears as the best thing since sliced bread for doing one particular thing, but may completely ignore others. for example, big research labs often have a very high computational budget, and focus on developing new architectures that may consume an obscenely high amount of compute resources to top a benchmark leaderboard by a fraction of a percent. Thankfully, many researchers now understand and appreciate that not everyone can fit a supercomputing cluster in their two-bedroom studio apartment, and there is an increasing interest in creating small, lightweight models.

But aprt from this, Transformers are increasingly being used in other domains, where they might not seem like a great fit initially. One of these is computer vision.

## Transformers for Computer Vision

## Conclusion

So there you have it-a deep dive into the transformer architecture. At this stage in the book, we hope that you are starting to get some idea of what deep learning researchers and engineers today are thinking about. You might also start to come up with ideas of your own. Usually, these start simple-something along the lines of "Gee, this attention mechanism doesn't fit on my GPU. I wonder if I can generate a smaller set of matrices in some way." We highly encourage you to try these out! Often, these simple ideas, after iteration and testing, are what lead people to create breakthrough research idears and revolutionary new products.

Here's a quick summary of the key ideas in this chapter:

- Transformers were first proposed in the 2017 paper "Attention Is All You Need" by Vaswani et al.
- Transfomers remove the recurrent portion of the RNN architecture and use only an attention mechanism, allowing them to be parallelized across sentences.
- Attention mechanisms are a type of layer in a neural network that allows them to collect and combine "global features" (information from every point in a large input sequence).
- Attention mechanisms come in many flavors and are used across many domains and architectures, not just transformers.
- The standard attention mechanism used in the Transformer architecture is called Multi-Head Self-Attention(MHSA). It transforms the input into a small key space and repeats the dot product attention multiple times.
- Attention mechanisms are very powerful but are also computationally expensive. The standard MHSA has an $n^2$ memory cost, which means that if you have 10 words in your sentence, you need to store 10*10 = 100 attention weights.
- The attention weights between $x$ and $y$ can be interpreted as "how much are $x$ and $y$ related?" in an abstract sense (usefull in pronoun resolution).
- Attention weights can be a useful visualization tool.
- There is significant research being conducted in assessing how to build a new, more computationally efficient attention mechanism. So far, there is no clear best approach, and most practitioner still use MHSA for simplicity.

While this chapter is now coming to an end, the story of transformers is not. Next, we'll look at the sequence of events that fed the explosive growth of NLP in the last several years. Transformers played a huge role here, and new models like BERT, RoBERTa, and GPT-3 will show you how we can take this simple idea of an attention mechanism, scale it up, and create incredibly powerful NLP mdoels.

# Fine-tuning BERT for sentiment analysis

## Contents
1. 학습데이터 확인
2. 보조함수 확인
3. SenitmentClassifier 확인
4. Analyser 확인
5. 학습 과정 확인
6. tests

## 학습 데이터

In [ ]:
!pip3 install torch
!pip3 install transformers
from typing import List, Tuple
from transformers import BertTokenizer, BertModel
import torch
from torch.nn import functional as F


DATA: List[Tuple[str, int]] = [
    # 긍정적인 문장 - 1
    ("난 너를 좋아해", 1),
    # --- 부정적인 문장 - 레이블 = 0
    ("난 너를 싫어해", 0)
]

TESTS = [
    "나는 자연어처리가 좋아",
    "나는 자연어처리가 싫어",
    "나는 너가 좋다",
    "너는 참 좋다",
]

## 보조함수

In [ ]:
# cuda를 사용할 수 있는지를 체크, 사용가능하다면 cuda로 설정된 device를 출력.
def load_device() -> torch.device:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    return device


# 텐서를 구축하는 부분 - X
def build_X(sents: List[str], tokenizer: BertTokenizer, device: torch.device) -> torch.Tensor:
    """
    return X (N, 3, L).  N, 0, L -> input_ids / N, 1, L -> token_type_ids / N, 2, L -> attention_mask
    """
    encodings = tokenizer(text=sents,
                          add_special_tokens=True,
                          return_tensors='pt',
                          truncation=True,
                          padding=True)
    return torch.stack([
        encodings['input_ids'],
        encodings['token_type_ids'],
        encodings['attention_mask']
    ], dim=1).to(device)

# 텐서를 구축하는 부분 - y
def build_y(labels: List[int], device: torch.device) -> torch.Tensor:
    return torch.FloatTensor(labels).unsqueeze(-1).to(device)

## Sentiment Classifier

In [ ]:
class SentimentClassifier(torch.nn.Module):
    def __init__(self, bert: BertModel):
        super().__init__()
        self.bert = bert
        self.hidden_size = bert.config.hidden_size
        # TODO 1
        self.W_hy = torch.nn.Linear(..., ...)

    def forward(self, X: torch.Tensor) -> torch.Tensor:
        """
        :param X: (N, 3, L)
        :return: H_all (N, L, H)
        """
        input_ids = X[:, 0]
        token_type_ids = X[:, 1]
        attention_mask = X[:, 2]
        H_all = self.bert(input_ids, token_type_ids, attention_mask)[0]
        return H_all

    def predict(self, X: torch.Tensor) -> torch.Tensor:
        """
        :param X:
        :return:
        """
        # TODO 2
        H_all = self.forward(X)
        H_cls = ...
        y_hat = ...  # (N, H) * (H, 1) -> (N, 1)
        return torch.sigmoid(y_hat)

    def training_step(self, X: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
        # TODO 3
        y_hat = self.predict(X)
        loss = ...
        return loss.sum()

## Analyser

In [ ]:

class Analyser:
    """
    Bert 기반 감성분석기.
    """
    def __init__(self, classifier: SentimentClassifier, tokenizer: BertTokenizer, device: torch.device):
        self.classifier = classifier
        self.tokenizer = tokenizer
        self.device = device

    def __call__(self, text: str) -> float:
        X = build_X(sents=[text], tokenizer=self.tokenizer, device=self.device)
        y_hat = self.classifier.predict(X)
        return y_hat.item()

## Training

In [ ]:
# 사전학습된 버트 모델을 로드
tokenizer = BertTokenizer.from_pretrained('beomi/kcbert-base')
bert = BertModel.from_pretrained('beomi/kcbert-base')

# --- have a look at the config --- #
print(bert.config)
print(bert.config.hidden_size)

In [ ]:
# --- hyper parameters --- #
EPOCHS = 20
LR = 0.0001


device = load_device()
print(device)

# --- build the dataset --- # 
sents = [sent for sent, _ in DATA]
labels = [label for _, label in DATA]
X = build_X(sents, tokenizer, device)
y = build_y(labels, device)

# --- instantiate the classifier --- #
classifier = SentimentClassifier(bert)
classifier.to(device)  # 모델도 gpu에 올리기. 
optimizer = torch.optim.Adam(classifier.parameters(), lr=LR)  # 최적화 알고리즘을 선택.

# --- 학습시작 --- #
for epoch in range(EPOCHS):
    loss = classifier.training_step(X, y)
    loss.backward()  # 오차 역전파
    optimizer.step()  # 경사도 하강
    optimizer.zero_grad()  # 기울기 축적방지
    print(f"epoch:{epoch}, loss:{loss.item()} ")

## Test

In [ ]:
classifier.eval()
analyser = Analyser(classifier, tokenizer, device)

for sent in TESTS:
    print(sent, "->", analyser(sent))

## Conclusion